add user into db by nickname

add columns to users table: https://boardgamegeek.com/user/zombiegod
- work link (boolean)
- Registration Date (date)
- Last Profile Update (date)
- Last Login (date)

aldoritm if we dont have user in db: 
- check link
- get profile info
- insert user into db

In [125]:
import requests as re
from bs4 import BeautifulSoup
import sqlite3
from datetime import date

import pandas as pd
import numpy as np

import time
from tqdm.notebook import tqdm

PATH_TO_DB = '../Board-Game-Data/board_game_database.db'

In [ ]:
ratings = pd.read_feather('../Board-Games/data/full/bgg_ratings_full.feather')

In [129]:
def clear_link(link: str) -> str:
    if link.find('://') >= 0:
        link = link[link.find('://')+3:]
    if link[:4] == 'www.':
        link = link[4:]
    return link.strip("/")

def get_user_profile(nickname: str) -> dict:
    '''
    Get dict with user profile data from boardgamegeek.com
    '''
    url_main = 'https://boardgamegeek.com/user/'

    r = re.get(url_main + nickname)
    soup = BeautifulSoup(r.text)
    table = soup.find('table', class_='profile_table')
    rows = table.find_all('tr')
    user_dict = {}
    for row in rows:
        info = row.find_all('td')
        user_dict[info[0].text[:-1].lower().replace(' ', '_').split('/')[-1]] = info[1].text.strip()
    user_dict['website'] = clear_link(user_dict['website'])
    if re.get('http://' + user_dict['website']).status_code != 200:
        user_dict['website'] = ''
    return user_dict


# user_id INTEGER PRIMARY KEY,
# nickname TEXT NOT NULL UNIQUE,
# country_id INTEGER REFERENCES сountries(country_id),
# website_id INTEGER REFERENCES links(link_id),
# registration_date DATE #NOT NULL
# last_profile_update DATE #NOT NULL
# last_login DATE #NOT NULL
# last_check DATE DEFAULT CURRENT_DATE NOT NULL
def add_user_into_db(nickname: str, PATH_TO_DB: str):
    '''
    Function check user nickname in db, and addd info about this user if needed
    ==========
    Parameters:
    
    nickname: str - user nackname, his/her profile will be checked on boardgamegeek.com
    PATH_TO_DB: str - path to local bg_database (sqlite3)
    '''
    conn = sqlite3.connect(PATH_TO_DB)

    # create a cursor object
    cursor = conn.cursor()
    commit_flag = False
    
    # checking nickname in database
    cursor.execute('''
                    SELECT user_id
                    FROM users 
                    WHERE nickname = ?
                    ''',(nickname, ))
    if cursor.fetchall() == []:
        flag = True
    else: 
        flag = False
    
    
    if flag:
        try:
            user_dict = get_user_profile(nickname)

            # get country code
            country_id = np.nan
            if user_dict['country'] != '':
                cursor.execute('''
                            SELECT country_id
                            FROM countries 
                            WHERE country = ?
                            ''',(user_dict['country'], ))
                country_id = cursor.fetchall()
                if country_id != []:
                    country_id = country_id[0][0]
                else:
                    print('### No country in DB: ', user_dict['country'])   

            website_id = np.nan
            if user_dict['website'] != '':
                cursor.execute('''
                                SELECT link_id
                                FROM links 
                                WHERE link = ?
                                ''',(user_dict['website'], ))
                website_id = cursor.fetchall()
                if website_id == []:
                    cursor.execute('''
                        INSERT INTO links (link) VALUES (?)
                        ''',(user_dict['website'], ))
                    print('Add link to db -> ', user_dict['website'])
                    cursor.execute('''
                                SELECT link_id
                                FROM links 
                                WHERE link = ?
                                ''',(user_dict['website'], ))
                    website_id = cursor.fetchall()[0][0]
    #             website_id = website_id[0][0]

            last_check = date.today()
#             print(nickname, country_id, website_id, user_dict['registration_date'], user_dict['last_profile_update'], user_dict['last_login'], last_check)
            cursor.execute('''
                INSERT INTO users (nickname, country_id, website_id, registration_date, last_profile_update, last_login, last_check) VALUES (?, ?, ?, ?, ?, ?, ?)
                ''',(nickname, country_id, website_id, user_dict['registration_date'], user_dict['last_profile_update'], user_dict['last_login'], last_check))
            commit_flag = True
        except: 
            print(f'### Check {nickname} manually')
    if commit_flag:
        # commit the changes
        conn.commit()
#         print('Changes in DB was commited')

    # close the connection
    conn.close()
        

In [88]:
add_user_into_db('zombiegod', PATH_TO_DB=PATH_TO_DB)

In [109]:
user = get_user_profile('DiVo1975')
user

{'registration_date': '2015-01-01',
 'last_profile_update': '2019-05-30',
 'last_login': '2023-03-22',
 'country': '',
 'state': '',
 'city': '',
 'website': '',
 'geekmail': 'Send Private Message to DiVo1975'}

In [104]:

ratings

,nickname,title,boardgame_id,rating,num_of_plays,comment,own,prevowned,fortrade,want,wanttoplay,wanttobuy,wishlist,preordered,last_modified
0,DiVo1975,Anachrony,185343,9.0,2,None,0,1,0,0,0,0,0,0,2022-11-06 14:02:30
1,DiVo1975,Anachrony: Infinity Box,278292,10.0,2,None,1,0,0,0,0,0,0,0,2022-01-06 06:21:59
2,DiVo1975,Ark Nova,342942,10.0,25,None,1,0,0,0,0,0,0,0,2022-09-05 01:21:16
3,DiVo1975,Arkham Horror: The Card Game,205637,9.0,0,None,1,0,0,0,0,0,0,0,2022-11-06 14:02:55
4,DiVo1975,Blood Rage,170216,10.0,1,None,1,0,0,0,0,0,0,0,2022-11-06 14:03:59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20186643,zinnwen,Taj Mahal,475,7.0,0,None,0,0,0,0,0,0,0,0,2009-05-25 03:54:26
20186644,zinnwen,Through the Ages: A New Story of Civilization,182028,9.0,0,None,0,0,0,0,0,0,0,0,2017-09-01 11:38:04
20186645,zinnwen,Ticket to Ride,9209,6.0,0,None,0,0,0,0,0,0,0,0,2009-05-25 03:54:31
20186646,zinnwen,Tigris & Euphrates,42,8.0,0,None,0,0,0,0,0,0,0,0,2009-05-25 03:53:41


In [105]:
conn = sqlite3.connect(PATH_TO_DB)

# create a cursor object
cursor = conn.cursor()

cursor.execute('''
            SELECT *
            FROM users
            WHERE nickname = ?
            ''',('DiVo1975', ))
data = cursor.fetchall()
       

# cursor.execute('''
#             INSERT INTO descriptions (boardgame_id, description) VALUES (?, ?)
#             ''',(int(row.boardgame_id), row.description))

# commit the changes
conn.commit()

# close the connection
conn.close()
data

[]

In [127]:
conn = sqlite3.connect('../Board-Game-Data/board_game_database.db')
cursor = conn.cursor()
# i = 'Joachim Zischke | terra authentica'
check = []
for nick in list(ratings.nickname.unique()):
#     i = 'zombiegod'

    # cursor.execute('INSERT INTO publishers (publisher) VALUES (?)', (i,))
    cursor.execute('''
    SELECT *
    FROM users
    WHERE nickname = ?
    ''',(nick,))


    data = cursor.fetchall()
    if data == []:
        check.append(nick)
# conn.commit()
conn.close()
# data
len(check)

4118

In [130]:
len(check)

4118

In [131]:
for i in tqdm(check):
    add_user_into_db(i, PATH_TO_DB=PATH_TO_DB)
    time.sleep(0.33)
#     break

  0%|          | 0/4118 [00:00<?, ?it/s]

Check horstderadler manually
Add link to db ->  http://www.geekgirlauthority.com/author/fenirob-2/
Add link to db ->  http://www.kassel-spielt.de
Check FaolanLuna manually
Add link to db ->  https://saxondogwarband.blogspot.com/
Add link to db ->  https://www.startnext.com/fussball-strategie-brettspiel
Add link to db ->  http://blog.drewsgameroom.com/
Add link to db ->  https://www.quora.com/profile/Bruno-Curfs
Add link to db ->  www.polaris.games
Add link to db ->  www.aspermyability.com
Add link to db ->  Chezspiel.com
Add link to db ->  www.mirrorboxgames.com
Add link to db ->  http://bgpov.com/
Check Glasgow17 manually
Add link to db ->  http://www.calthaer.com
Add link to db ->  www.archonagames.com
Add link to db ->  https://www.facebook.com/aliplaysalot/
Add link to db ->  https://www.youtube.com/channel/UC7B9dfgM-6zoyBq46oedMSQ
Add link to db ->  miniatures.watkins-family.co.uk
Add link to db ->  http://www.gameformstudios.com/
Add link to db ->  www.bicologie.com/theflyingchau